<a href="https://colab.research.google.com/github/dlsun/Stat425F19/blob/master/Numerical_Maximum_Likelihood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import Symbulate
!pip install -q symbulate
from symbulate import *

In [0]:
#@title Define `plot_continuous_function`
import matplotlib.pyplot as plt

def plot_continuous_function(f, xlim=(0, 1), xlabel=r"$\theta$", ylabel="Likelihood"):
  xs = np.linspace(xlim[0], xlim[1], 1000)
  ys = [f(x) for x in xs]
  plt.plot(xs, ys, "-")
  plt.xlabel(xlabel, fontsize=18)
  plt.ylabel(ylabel, fontsize=18)
  plt.xlim(*xlim)

# Numerical Maximum Likelihood

## Motivating Example

Let $X_1, ..., X_n$ be i.i.d. $\text{Gamma}(r, \lambda=1)$. What is the MLE of $r$?



First, we write down the likelihood and simplify as much as possible:
\begin{align}
L_{\vec x}(r) &= \prod_{i=1}^n \frac{1}{\Gamma(r)} x_i^{r-1} e^{-x_i} \\
&= \frac{1}{\Gamma(r)^n} \left(\prod_{i=1}^n x_i \right)^{r-1} e^{-\sum_{i=1}^n x_i}.
\end{align}

Now let's calculate the log likelihood:
$$ \ell(r) = -n\log \Gamma(r) + (r-1) \log \prod_{i=1}^n x_i - \sum_{i=1}^n x_i. $$

Finally, we take the derivative to obtain the score equation:
$$ \ell'(r) = -n \frac{\Gamma'(r)}{\Gamma(r)} + \log \prod_{i=1}^n x_i = 0. $$

Solving this equation by hand is impossible. Remember that $\Gamma(r)$ has a complicated definition: $\int_0^\infty t^{r-1} e^{-t}\,dt$. Now we have to take its derivative?!

In [0]:
# Simulate some data from a gamma distribution,
# where r=4.5.
X = RV(Gamma(shape=4.5, rate=1))
x = X.sim(10)
x

Now let's graph the log-likelihood for this data.

In [0]:
from scipy.special import gamma

def log_likelihood(r):
  n = len(x)
  return -n * log(gamma(r)) + (r-1) * sum(log(x)) - sum(x)

plot_continuous_function(log_likelihood, 
                         xlim=(0, 10),
                         xlabel="$r$",
                         ylabel="Log Likelihood")

We can find the MLE by eyeballing it. But can we make a computer do it?

# Newton's Method

Newton's method is a technique for finding the roots of a function. That is, it is a way to find values $x$ such that

$$ f(x) = 0. $$

For example, suppose 

$$ f(x) = x (2 x - 1), $$

graphed below.

In [0]:
def f(x):
  return x * (2 * x - 1)

plot_continuous_function(f)
plt.hlines(y=0, xmin=0, xmax=1)

We know that its roots should be $x=0$ and $x=1/2$. But what if we did not know this? Could we start with a blind guess and improve our guess until we get the right answer?

For example, suppose we start with the guess $x_0 = 0.8$. Clearly, $f(0.8) \neq 0$. How would we improve on this guess? We can find a tangent line to the function at $x=0.8$ and find where that tangent line crosses the $x$-axis. This procedure is shown graphically below.

In [0]:
def f(x):
  return x * (2 * x - 1)

def df(x):
  return 4 * x - 1

plot_continuous_function(f)
plt.hlines(y=0, xmin=0, xmax=1)

plt.vlines(x=0.8, ymin=0, ymax=f(0.8), linestyles="dashed")

xs = np.arange(0.4, 1.1, step=0.1)
plt.plot(xs, f(0.8) + df(0.8) * (xs - 0.8), 'r-')

Note that the equation of this tangent line is 

$$ t(x) = f(x_0) + f'(x_0) (x - x_0). $$

We find where this tangent line crosses the $x$-axis. In other words, we find $x$ such that 

$$ t(x) = 0. $$

The solution is 

$$ x = x_0 - \frac{f(x_0)}{f'(x_0)}. $$

This value becomes our updated guess $x_1$. Now, we repeat this process:

- Find the tangent line to the curve at $x_t$.
- Find where this tangent line crosses the $x$-axis.
- This location is the new guess $x_{t+1}$.

This produces iterates $x_2$, $x_3$, ... that are successively closer to the true root of $0.5$.

Let's run 10 iterations of Newton's method, starting with an initial guess of $x=0.8$.

In [0]:
# initial guess
x = 0.8

# 10 iterations of Newton's method
for i in range(10):
  print("Iteration %d: %.10f" % (i, x))
  x = x - f(x) / df(x)

print(x)

Newton's method is remarkable. It converges to the right answer extremely quickly, usually within 5 iterations.

# Applying Newton's Method to MLEs

Use Newton's method to find the MLE of $r$ in the gamma model, using the data you generated above.

Remember that the MLE solves the score equation 
$$ \ell'(r) = 0. $$
So we apply Newton's method to the function $f \equiv \ell'$. (Since Newton's method also requires $f'$, we will also need to calculate $\ell''$.)

_Hint:_ The [digamma function](https://en.wikipedia.org/wiki/Digamma_function) will come in handy when you implement the score $\ell'$. For the derivative of the score $\ell''$, you'll need the derivative of the digamma function, which is provided by the [polygamma function](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.special.polygamma.html). (Note that `polygamma(0, x)` is equivalent to `digamma(x)`.)

In [0]:
from scipy.special import gamma, digamma, polygamma

# We use the data x defined above.
# Define n to be the number of observations.
n = len(x)

def score(r):
  # TODO: Implement the score: the derivative of the log-likelihood.
  # This is analogous to f above. It's the function we want the root of.
  return r

def d_score(r):
  # TODO: Implement the derivative of the score.
  # This is analogous to df above. It is the slope of the tangent line.
  return 1

# Run 10 iterations of Newton's method
r = 1 # Start with an initial bad guess of r.
for i in range(10):
  # TODO: Implement Newton's method.
  continue

# Newton's Method in Higher Dimensions

Newton's method is not necessary for finding roots of one-dimensional functions, since we can simply graph the function and find the root by zooming in on the graph.

The power of Newton's method is fully realized for multidimensional function, when the function is difficult or even impossible to visualize. In higher dimensions, Newton's method tries to find $\vec x$ such that 

$$ \vec f(\vec x) = \vec 0. $$

Recall that in one dimension, Newton's method says that we should start with an initial guess $x$ and repeatedly update $x$ as follows:

$$ x \leftarrow x - f'(x)^{-1} f(x). $$

This rule generalizes to higher dimensions as follows. The "derivative" $f'$ becomes a matrix of partial derivatives $[D\vec f(\vec x)]$, called [the Jacobian](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant), and the update rule is 

$$ \vec x \leftarrow \vec x - [D\vec f(\vec x)]^{-1} \vec f(\vec x). $$

The inverse is a matrix inverse, and the multiplication is matrix multiplication.

You will see an application and an implementation of this multivariate Newton's method on the lesson for tomorrow, when we apply this to logistic regression.